In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import ast

import re

pd.options.display.max_columns = 300
pd.options.display.max_colwidth = 200
pd.options.display.min_rows = 200
pd.options.display.max_rows = 200


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filePath = os.path.join(dirname, filename)
        fileCsv = pd.read_csv(filePath)
        
        display(filePath)
        display(fileCsv.head())
        display(fileCsv.shape)

        variable = Path(filename).stem
        exec(f'data_{variable} = fileCsv')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Objectives  

`id = {00016_001}`
Given the patient (pn_num=16)(look for the corresponding patient history notes), with specified feature needed (feat_num=001). Identify the target `position`, which is the (start, stop) index of the string that is related to the interested feature.  

# EDA

In [ ]:
a = data_features.groupby('case_num').feature_text.apply(lambda series: series.value_counts()).reset_index()
display(a.head())
a.shape
# the number of features is still 143 rows. 
# this suggest one feature cannot appear in more than 1 case
# thus case_num can be used to select the candiate for the features

In [ ]:
# the count of features in each case_num
# looks almost evenly distributed
data_features.groupby('case_num').feature_text.apply(len)

## `patient_notes.csv` and `train.csv`
There are 42,146 notes, and 14,300 rows of labelled data

In [ ]:
'unique patients in notes', len(data_patient_notes.pn_num.unique())

In [ ]:
'patients in train data', len(data_train.pn_num.unique()) # 1000 patients in the training notes

In [ ]:
# can a patient be in different case?
'number of patients having more than one case', sum(data_train.groupby('pn_num').case_num.apply(lambda series: len(series.unique())) > 1)

In [ ]:
patientInTrain = data_train.pn_num
data_patient_notes.pn_num.isin(patientInTrain).sum() # all thousand notes found
# with 14k notes, only 1000 notes comes with information annotation

In [ ]:
display(data_patient_notes.head())
display(data_patient_notes.shape)


data_train = pd.merge(data_train, data_features,
                      on='feature_num', how='left',
                      suffixes=('', '_duplicate')
                     )
data_train = data_train.loc[:,~data_train.columns.str.endswith('_duplicate')]
display(data_train.head())
display(data_train.shape)

In [ ]:
data_train = pd.merge(data_train, data_patient_notes,
                     on=['pn_num', 'case_num'])

In [ ]:
print(data_train.shape, 'total number of data entry')
print(len(data_train.pn_history.unique()), 'unique number of patient notes')

EDA Questions:
* for each note(patient), how many labels are present in the text
* for each note(patient), what is the len of the text(char and token count)
* for every annotation, what is the distribution of char/token count of the text (some feature might need more words to describe)

In [ ]:
data_train['annotationParsed'] = data_train.annotation.apply(lambda anno: ast.literal_eval(anno))

In [ ]:
data_train[data_train.location.str.contains(';')].head()

## location explained

for each `pn_history` note, 

In [ ]:
data_train['locationParsed'] = data_train.location.str.replace(';', "','")
data_train['locationParsed'] = data_train.locationParsed.apply(lambda anno: ast.literal_eval(anno))
data_train['locationParsed'] = data_train['locationParsed'].apply(
lambda listedStrings: list(tuple(item.split(' ')) for item in listedStrings)
)
data_train['locationParsed'] = data_train['locationParsed'].apply(
lambda listed_StringTuple: list((int(a[0]), int(a[1])) for a in listed_StringTuple) if listed_StringTuple else []
)

In [ ]:
annoByPatient = data_train.groupby(['pn_num']).annotationParsed

In [ ]:
# number of labels per note/patient
annoByPatient.apply(
lambda series: sum(series.apply(len)>=1)
).plot.hist(bins=20)